In [1]:
import pandas as pd  # Data handling
import numpy as np   # Math
import matplotlib.pyplot as plt  # Charts
import seaborn as sns  # Better charts
from sklearn.linear_model import LinearRegression  # ML
import statsmodels.api as sm  # ARIMA
from prophet import Prophet  # Forecasting (if needed, Colab has it)
import sqlite3  # Database
import openpyxl  # Excel reports
!pip install reportlab
from google.colab import drive
drive.mount('/content/drive')
#from reportlab.lib.pagesizes import letter  # PDF reports
#from reportlab.pdfgen import canvas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.1 MB/s eta 0:00:00
Mounted at /content/drive


In [2]:
df1 = pd.read_csv('/content/drive/MyDrive/Data_Analysis_Project_paython_sem_6/clean_data/retail_sales_clean_2.csv',parse_dates=['Date'])
df2 = pd.read_csv('/content/drive/MyDrive/Data_Analysis_Project_paython_sem_6/clean_data/retail_store_clean_2.csv',parse_dates=['Date'],dayfirst=True)
df3 = pd.read_csv('/content/drive/MyDrive/Data_Analysis_Project_paython_sem_6/clean_data/superstore_clean_2.csv', parse_dates=['Order Date', 'Ship Date'],dayfirst=True, date_format='mixed')  # jo column date wale hain unke naam likh do
df2['Date'] = pd.to_datetime(df2['Date'], dayfirst=True, errors='coerce')


In [3]:
import pandas as pd
from prophet import Prophet

# 1. सारे प्रोडक्ट्स की लिस्ट निकालो
product_list = df2['Product ID'].unique()
print(f"Total Products found: {len(product_list)}")

all_forecasts = []  # खाली बक्सा जहाँ हम रिजल्ट डालेंगे

# 2. लूप शुरू (हर प्रोडक्ट के लिए मशीन चलेगी)
# (नोट: अगर बहुत सारे प्रोडक्ट हैं तो इसमें टाइम लग सकता है)
for pid in product_list:
    # a. सिर्फ इस प्रोडक्ट (pid) का डेटा निकालो
    df_p = df2[df2['Product ID'] == pid].copy()

    # b. डेटा को सही फॉर्मेट में लाओ (Date aur Units Sold)
    df_p = df_p.groupby('Date')['Units Sold'].sum().reset_index()
    df_p.columns = ['ds', 'y']  # Prophet को ds aur y नाम ही चाहिए

    # (Check: अगर डेटा बहुत कम है (2 दिन से कम), तो इसे छोड़ दो)
    if len(df_p) < 2:
        continue

    # c. मॉडल ट्रेन करो
    # (print हटा दिया ताकि स्क्रीन न भरे, बस काम होने दो)
    m = Prophet(yearly_seasonality=True, daily_seasonality=False)
    m.fit(df_p)

    # d. अगले 30 दिन की भविष्यवाणी (Forecast)
    future = m.make_future_dataframe(periods=30)
    forecast = m.predict(future)

    # e. सिर्फ भविष्य (Future) का डेटा निकालो
    last_date = df_p['ds'].max()
    future_forecast = forecast[forecast['ds'] > last_date].copy()

    # f. प्रोडक्ट का नाम वापस जोड़ो ताकि पता चले ये किसका डेटा है
    future_forecast['Product ID'] = pid

    # g. 'yhat' मतलब 'Predicted Demand'. हम इसे लिस्ट में डाल देंगे
    all_forecasts.append(future_forecast[['ds', 'Product ID', 'yhat']])

# 3. सारे रिजल्ट्स को जोड़कर एक फाइनल टेबल बनाओ
final_reorder_list = pd.concat(all_forecasts)

print("\n✅ काम हो गया भाई! Reorder List तैयार है।")
print(final_reorder_list.head())

Total Products found: 20

✅ काम हो गया भाई! Reorder List तैयार है।
            ds Product ID        yhat
289 2024-01-02      P0001  544.637840
290 2024-01-03      P0001  493.467895
291 2024-01-04      P0001  566.196585
292 2024-01-05      P0001  501.773750
293 2024-01-06      P0001  514.156959


In [15]:
# ---------------------------------------------------------
# Step 1: Calculate Total Demand (Prediction)
# ---------------------------------------------------------
# Prophet के रिजल्ट से अगले 30 दिन की डिमांड जोड़ रहे हैं
demand_summary = final_reorder_list.groupby('Product ID')['yhat'].sum().reset_index()
demand_summary.rename(columns={'yhat': 'Predicted_Demand'}, inplace=True)

# डिमांड को राउंड-ऑफ कर रहे हैं
demand_summary['Predicted_Demand'] = demand_summary['Predicted_Demand'].round().astype(int)

# ---------------------------------------------------------
# Step 2: Calculate Current Stock (From df2 - Latest Value) ✅
# ---------------------------------------------------------
# df2 में 'Inventory Level' है, हम हर प्रोडक्ट का 'Latest Stock' (आखिरी तारीख का) उठाएंगे
# क्योंकि पुराने स्टॉक को जोड़ने का मतलब नहीं है।

# पहले डेटा को तारीख के हिसाब से सॉर्ट करें
df2_sorted = df2.sort_values('Date')

# अब हर Product ID का सबसे आखिरी (Latest) रिकॉर्ड निकालें
current_stock = df2_sorted.groupby('Product ID').tail(1)[['Product ID', 'Inventory Level']]
current_stock.rename(columns={'Inventory Level': 'Current_Stock'}, inplace=True)

# ---------------------------------------------------------
# Step 3: Merge Demand & Stock
# ---------------------------------------------------------
final_report = pd.merge(demand_summary, current_stock, on='Product ID', how='left')

# अगर कोई प्रोडक्ट स्टॉक लिस्ट में नहीं मिला, तो उसका स्टॉक 0 मानेंगे
final_report['Current_Stock'] = final_report['Current_Stock'].fillna(0).astype(int)

# प्रोडक्ट के नाम/कैटेगरी भी जोड़ लेते हैं (df2 से)
product_info = df2[['Product ID', 'Category']].drop_duplicates()
final_report = pd.merge(final_report, product_info, on='Product ID', how='left')

# ---------------------------------------------------------
# Step 4: Calculate Actual Reorder Quantity
# ---------------------------------------------------------
# Formula: Demand - Stock = Reorder
final_report['Reorder_Quantity'] = final_report['Predicted_Demand'] - final_report['Current_Stock']

# अगर माइनस में आए (मतलब स्टॉक ज्यादा है), तो उसे 0 कर दो
final_report['Reorder_Quantity'] = final_report['Reorder_Quantity'].apply(lambda x: x if x > 0 else 0)

# ---------------------------------------------------------
# Step 5: Show & Save Result
# ---------------------------------------------------------
# सबसे ज्यादा मंगाने वाले आइटम ऊपर
final_report = final_report.sort_values(by='Reorder_Quantity', ascending=False)

print("📋 FINAL CORRECTED REORDER LIST (Using Latest Inventory from df2):")
print(final_report[['Product ID', 'Category', 'Predicted_Demand', 'Current_Stock', 'Reorder_Quantity']].head(10))

# फाइल सेव करें
final_report.to_csv("Real_Reorder_List_Final.csv", index=False)
print("\n✅ File Saved: Real_Reorder_List_Final.csv")

📋 FINAL CORRECTED REORDER LIST (Using Latest Inventory from df2):
   Product ID     Category  Predicted_Demand  Current_Stock  Reorder_Quantity
85      P0018     Clothing             23296            151             23145
87      P0018         Toys             23296            151             23145
86      P0018    Groceries             23296            151             23145
89      P0018  Electronics             23296            151             23145
88      P0018    Furniture             23296            151             23145
50      P0011    Furniture             22867            156             22711
51      P0011    Groceries             22867            156             22711
53      P0011  Electronics             22867            156             22711
52      P0011         Toys             22867            156             22711
54      P0011     Clothing             22867            156             22711

✅ File Saved: Real_Reorder_List_Final.csv


In [16]:
from google.colab import files
files.download('Real_Reorder_List_Final.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>